In [1]:
# Libs
import os
import numpy as np
import json
from tqdm import tqdm
from operator import itemgetter
import pandas as pd

In [2]:
DATA_INPUT_PATH = 'data'
DATA_INPUT_NAME1 = 'support.json'
DATA_INPUT_NAME2 = 'dev.json'
DATA_INPUT_NAME3 = 'train.json'
DATA_INPUT_NAME4 = 'test.json'

In [3]:
# Data
with open(os.path.join(DATA_INPUT_PATH, DATA_INPUT_NAME1)) as file_json:
    data_twitter_support = json.load(file_json)
with open(os.path.join(DATA_INPUT_PATH, DATA_INPUT_NAME2)) as file_json:
    data_twitter_dev = json.load(file_json)
with open(os.path.join(DATA_INPUT_PATH, DATA_INPUT_NAME3)) as file_json:
    data_twitter_train = json.load(file_json)
with open(os.path.join(DATA_INPUT_PATH, DATA_INPUT_NAME4)) as file_json:
    data_twitter_test = json.load(file_json)

In [4]:
df_data_twitter_support = pd.DataFrame(data_twitter_support)
df_data_twitter_all = pd.DataFrame.from_records(data_twitter_train).append(pd.DataFrame.from_records(data_twitter_dev)).append(pd.DataFrame.from_records(data_twitter_test))


In [5]:
def retrieve_following(x):
    try:
        f = x['following']
        if f == []:
            f = [-999]
        else:
            f = f
    except:
        f = [-999]
    return f
def retrieve_follower(x):
    try:
        f = x['follower']
        if f == []:
            f = [-999]
        else:
            f = f
    except:
        f = [-999]
    return f

df_data_twitter_all['following'] = df_data_twitter_all['neighbor'].apply(retrieve_following)


In [7]:
# Let's create two structures
# 1) An array of two columns with source and target nodes (follow network)
# 2) An array 
array_source_target = np.empty((1,2))
array_features_nodes = np.empty((1, 8))
y = []
for _,user in tqdm(df_data_twitter_all.iterrows()):
    try:
        protected = user['profile']['protected']
        n_followers = user['profile']['followers_count']
        n_friends = user['profile']['friends_count']
        listed_count = user['profile']['listed_count']
        geo_enabled = user['profile']['geo_enabled']
        verified = user['profile']['verified']
        statuses_count = user['profile']['statuses_count']
        user_id = int(user['ID'])
        user_ = np.array([user_id])   
        features = np.array([user_id, protected, n_followers, n_friends, listed_count, geo_enabled, verified, statuses_count])
        array_features_nodes = np.vstack([array_features_nodes,features])
        y.append(user['label'])  
        following_ = user['following']
    except:
        continue

    if (following_!=[-999]):
        following = np.array(following_)
        #followers = np.array(followers_)
        array_user_following = np.hstack([np.broadcast_to(user_, shape=(len(following), len(user_))), following.reshape(-1,1)])
        #array_user_follower = np.hstack([np.broadcast_to(user_, shape=(len(followers), len(user_))), followers.reshape(-1,1)])
        array_source_target = np.vstack([array_source_target,array_user_following])
    else:
        continue

11826it [02:54, 67.82it/s]


In [8]:
df_source_target = pd.DataFrame(array_source_target, columns=['source', 'target'])
df_features_nodes = pd.DataFrame(array_features_nodes, columns=['ID', 'protected', 'followers_count', 'friends_count', 'listed_count', 'geo_enabled','verified', 'statuses_count'])

In [9]:
df_features_nodes = df_features_nodes.iloc[1:]
df_source_target = df_source_target.iloc[1:]
df_features_nodes['y']=y
#df_features_nodes_na = df_features_nodes.dropna()
#df_source_target_na = df_source_target.dropna()
df_features_nodes = df_features_nodes.replace({'False ':0, 'True ':1})
df_features_nodes = df_features_nodes.replace({'nan':'2'})
df_features_nodes = df_features_nodes.astype('int')
df_source_target = df_source_target.astype('int')

In [11]:
df_source_target

,source,target
1,1297437077403885568,170861207
2,1297437077403885568,23970102
3,1297437077403885568,47293791
4,1297437077403885568,29458079
5,1297437077403885568,17995040
...,...,...
105697,3385331674,256099282
105698,3385331674,870728839823060992
105699,3385331674,1623581762
105700,3385331674,593882143


In [ ]:
del df_features_nodes_na_na

In [283]:
# clases
class_values = ['0', '1']
# id para as classes
class_idx = {name: id for id, name in enumerate(class_values)}
# id para os papers
user_idx = {int(name): int(idx) for idx, name in enumerate(sorted(set(set(list(df_source_target_na['source']) + list(df_source_target_na['target'])))))}

In [60]:
len(list(filter(lambda x: (x=='0') | (x=='1'),  y)))

11826

In [62]:
len(df_features_nodes_na['ID'].unique())

229484

In [63]:
len(df_source_target['source'].unique()) + len(df_source_target['target'].unique())

216265

In [285]:
df_source_target_na['source1'] = df_source_target_na['source'].apply(lambda x: user_idx.get(x, np.nan))
df_source_target_na['target1'] = df_source_target_na['target'].apply(lambda x: user_idx.get(x, np.nan))
df_features_nodes_na['ID'] = df_features_nodes_na['ID'].apply(lambda x: user_idx.get(x, np.nan))

In [287]:
df_features_nodes_na_na = df_features_nodes_na.dropna()
df_features_nodes_na_na = df_features_nodes_na_na.astype('int')

In [208]:
#df_features_nodes_na_na = df_features_nodes_na.dropna()

In [239]:
len(df_source_target)

227819

In [320]:
list_indices_filled_target = df_features_nodes_na_na[df_features_nodes_na_na['y']!=2]['ID'].values
y = df_features_nodes_na_na[df_features_nodes_na_na['y']!=2]['y'].values

In [323]:
#np.random.seed(10)
# 80% treino
size = int(0.8*len(list_indices_filled_target))
train_indices = list_indices_filled_target[0:size]
test_indices = list_indices_filled_target[size:]
#list_indices_treino = np.random.choice(range(len(list_indices_filled_target)),  size)

In [324]:
y_train = y[0:size]
y_test = y[size:]

In [329]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Define graph, namely an edge tensor and a node feature tensor
edges = tf.convert_to_tensor(df_source_target_na[["source1", "target1"]])
node_features = tf.convert_to_tensor(df_features_nodes_na_na.sort_values("ID").iloc[:, 1:-1])

In [330]:
class GraphAttention(layers.Layer):
    def __init__(
        self,
        units,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

    def build(self, input_shape):

        self.kernel = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
        )
        self.kernel_attention = self.add_weight(
            shape=(self.units * 2, 1),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
        )
        self.built = True

    def call(self, inputs):
        node_features, edges = inputs

        # Linearly transform node features (node states)
        node_features_transformed = tf.matmul(node_features, self.kernel)

        # (1) Compute pair-wise attention scores
        node_features_expanded = tf.gather(node_features_transformed, edges)
        node_features_expanded = tf.reshape(
            node_features_expanded, (tf.shape(edges)[0], -1)
        )
        attention_scores = tf.nn.leaky_relu(
            tf.matmul(node_features_expanded, self.kernel_attention)
        )
        attention_scores = tf.squeeze(attention_scores, -1)

        # (2) Normalize attention scores
        attention_scores = tf.math.exp(tf.clip_by_value(attention_scores, -2, 2))
        attention_scores_sum = tf.math.unsorted_segment_sum(
            data=attention_scores,
            segment_ids=edges[:, 0],
            num_segments=tf.reduce_max(edges[:, 0]) + 1,
        )
        attention_scores_sum = tf.repeat(
            attention_scores_sum, tf.math.bincount(tf.cast(edges[:, 0], "int32"))
        )
        attention_scores_norm = attention_scores / attention_scores_sum

        # (3) Gather node states of neighbors, apply attention scores and aggregate
        node_features_neighbors = tf.gather(node_features_transformed, edges[:, 1])
        out = tf.math.unsorted_segment_sum(
            data=node_features_neighbors * attention_scores_norm[:, tf.newaxis],
            segment_ids=edges[:, 0],
            num_segments=tf.shape(node_features)[0],
        )
        return out


class MultiHeadGraphAttention(layers.Layer):
    def __init__(self, units, num_heads=8, merge_type="concat", **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.merge_type = merge_type
        self.attention_layers = [GraphAttention(units) for _ in range(num_heads)]

    def call(self, inputs):
        atom_features, pair_indices = inputs

        # Obtain outputs from each attention head
        outputs = [
            attention_layer([atom_features, pair_indices])
            for attention_layer in self.attention_layers
        ]
        # Concatenate or average the node states from each head
        if self.merge_type == "concat":
            outputs = tf.concat(outputs, axis=-1)
        else:
            outputs = tf.reduce_mean(tf.stack(outputs, axis=-1), axis=-1)
        # Activate and return node states
        return tf.nn.relu(outputs)

In [331]:
class GraphAttentionNetwork(keras.Model):
    def __init__(
        self,
        node_features,
        edges,
        hidden_units,
        num_heads,
        num_layers,
        output_dim,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.node_features = node_features
        self.edges = edges
        self.preprocess = layers.Dense(hidden_units * num_heads, activation="relu")
        self.attention_layers = [
            MultiHeadGraphAttention(hidden_units, num_heads) for _ in range(num_layers)
        ]
        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs):
        node_features, edges = inputs
        x = self.preprocess(node_features)
        for attention_layer in self.attention_layers:
            x = attention_layer([x, edges]) + x
        outputs = self.output_layer(x)
        return outputs

    def train_step(self, data):
        indices, labels = data

        with tf.GradientTape() as tape:
            # Forward pass
            outputs = self([self.node_features, self.edges])
            # Compute loss
            loss = self.compiled_loss(labels, tf.gather(outputs, indices))
        # Compute gradients
        grads = tape.gradient(loss, self.trainable_weights)
        # Apply gradients (update weights)
        optimizer.apply_gradients(zip(grads, self.trainable_weights))
        # Update metric(s)
        self.compiled_metrics.update_state(labels, tf.gather(outputs, indices))

        return {m.name: m.result() for m in self.metrics}

    def predict_step(self, data):
        indices = data
        # Forward pass
        outputs = self([self.node_features, self.edges])
        # Compute probabilities
        return tf.nn.softmax(tf.gather(outputs, indices))

    def test_step(self, data):
        indices, labels = data
        # Forward pass
        outputs = self([self.node_features, self.edges])
        # Compute loss
        loss = self.compiled_loss(labels, tf.gather(outputs, indices))
        # Update metric(s)
        self.compiled_metrics.update_state(labels, tf.gather(outputs, indices))

        return {m.name: m.result() for m in self.metrics}

In [332]:
y_treino = y_treino.astype('int')
y_test = y_test.astype('int')

In [333]:
# Define hyper-parameters
HIDDEN_UNITS = 10
NUM_HEADS = 5
NUM_LAYERS = 3
OUTPUT_DIM = 2

NUM_EPOCHS = 2
BATCH_SIZE = 256
VALIDATION_SPLIT = 0.1
LEARNING_RATE = 3e-1
MOMENTUM = 0.9

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.SGD(LEARNING_RATE, momentum=MOMENTUM)
accuracy_fn = keras.metrics.SparseCategoricalAccuracy(name="acc")
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_acc", min_delta=1e-5, patience=5, restore_best_weights=True
)

# Build model
gat_model = GraphAttentionNetwork(
    node_features, edges, HIDDEN_UNITS, NUM_HEADS, NUM_LAYERS, OUTPUT_DIM
)

# Compile model
gat_model.compile(loss=loss_fn, optimizer=optimizer, metrics=[accuracy_fn])

gat_model.fit(
    x=train_indices,
    y=y_treino,
    validation_split=VALIDATION_SPLIT,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[early_stopping],
    verbose=2,
)

_, test_accuracy = gat_model.evaluate(x=test_indices, y=y_test, verbose=0)

print("--" * 38 + f"\nTest Accuracy {test_accuracy*100:.1f}%")

ValueError: Data cardinality is ambiguous:
  x sizes: 8504
  y sizes: 6479
Make sure all arrays contain the same number of samples.